In [214]:
import pandas as pd

In [215]:
df1 = pd.read_csv('playground-series-s3e13 (1)/train.csv')
df2 = pd.read_csv('playground-series-s3e13 (1)/trainn.csv')

In [216]:
df2.iloc[:, -1] = df2.iloc[:, -1].str.replace(' ', '_')

In [217]:
df1_ = df1.drop(df1.columns[0], axis = 1)
df2_ = df2.iloc[:,0:-1].astype(float).join(df2.iloc[:,-1])

In [218]:
shuffling_factor = 0.9
num_rows = 900

# calculate the number of rows to take from each dataframe
df2_rows = int(num_rows * shuffling_factor)
df1_rows = num_rows - df2_rows

# take random samples of rows from each dataframe
df1_sample = df1_.sample(df1_rows)
df2_sample = df2_.sample(df2_rows, replace = True)

# concatenate the samples and shuffle the rows
df_final = pd.concat([df1_sample, df2_sample]).sample(frac=1).reset_index(drop=True)

In [219]:
df_final

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
1,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Malaria
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yellow_Fever
3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Chikungunya
4,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Chikungunya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
896,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Japanese_encephalitis
897,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Dengue
898,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Rift_Valley_fever


In [220]:
d = list(i for i in range(1,901))

df_final.insert(loc=0,column='id',value=d)

In [221]:
x,y = df_final.iloc[:,:-1],df_final.iloc[:,-1]

In [222]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [223]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB(min_categories=y.nunique())

cnb.fit(x_train,y_train)

CategoricalNB(min_categories=11)

In [224]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
y2 = y.values.reshape(-1,1)
enc.fit(y2)

OrdinalEncoder()

In [225]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [226]:
y_train

173                   Dengue
6                     Plague
581          West_Nile_fever
377    Japanese_encephalitis
73                    Dengue
               ...          
106    Japanese_encephalitis
270    Japanese_encephalitis
860             Lyme_disease
435    Japanese_encephalitis
102          West_Nile_fever
Name: prognosis, Length: 603, dtype: object

In [227]:
params = {
    'alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}

In [228]:
from sklearn.metrics import make_scorer
scorer = make_scorer(mapk,greater_is_better=True)

In [229]:
x_train.head()


,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
173,174,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,7,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
581,582,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
377,378,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,74,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [230]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(estimator=cnb,param_grid=params,cv=5,n_jobs=-1, scoring=scorer)
cv.fit(x_train,y_train)
y_pred = cv.predict_proba(x_test)
sort_pred = np.argsort(-y_pred, axis=1)
top_3 = sort_pred[:,:3]

og = top_3.shape
top_3=enc.inverse_transform(top_3.reshape(-1,1))
top_3=top_3.reshape(og)

print(mapk(y_test.values.reshape(-1,1), top_3, k=3))


0.9203142536475869


c:\Users\19199\.conda\envs\ML\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [231]:
test = pd.read_csv('playground-series-s3e13 (1)/test.csv')

y_pred = cv.predict_proba(x_test)
sort_pred = np.argsort(-y_pred, axis=1)
top_3 = sort_pred[:,:3]

og = top_3.shape
top_3=enc.inverse_transform(top_3.reshape(-1,1))
top_3=top_3.reshape(og)

x_test['prognosis'] = np.apply_along_axis(lambda x: np.array(' '.join(x), dtype="object"), 1, top_3)
x_test.to_csv('playground-series-s3e13 (1)/submission.csv', columns=['id', 'prognosis'], index=False)